In [ ]:
import sys
sys.path.append('../')
from src.streetview import save_streetview_image, KEY
import pandas as pd
import numpy as np
import re
import time
from django.utils import text
from os import listdir
from os.path import isfile, join

In [ ]:
# df = pd.read_csv('../data/street_view_sample/sample_residences_102219.csv')

In [ ]:
def clean_address(adr, slugify = True):
    street_adr = adr.split(',')[0]
    fn = re.sub(' +','_',street_adr)
    if slugify:
        fn = text.slugify(fn)
    return fn + '.jpg'

In [ ]:
# for address in df.ADDRESS_FOR_STREET_VIEW:
#     filename = f"../data/street_view_sample/{clean_address(address)}.jpg"
#     save_streetview_image(address, filename)

# Clean Data

In [ ]:
df = pd.read_excel('../data/residence_addresses_googlestreetview.xlsx', sheet_name = 3)

print(len(df))
df = df[['MBL','SITE_ADDR']]

df.SITE_ADDR = df.SITE_ADDR.apply(lambda x: x.split('#')[0].strip().replace('  ',' '))

df = df.drop_duplicates()

print(len(df))

address_list = df.groupby('MBL')['SITE_ADDR'].apply(list).reset_index()

print(len(address_list))

def clean_addr(addr_list):
    if len(addr_list) == 0:
        return np.nan
    elif len(addr_list) == 1:
        return addr_list[0]
    else:
        for addr in addr_list:
            if '-' in addr:
                return addr
            if re.search('\d \d', addr):
                return re.sub('\d \d', lambda x: x.group(0).replace(' ','-'), addr)
            else:
                return addr_list[0]

address_list.SITE_ADDR = (
    address_list.SITE_ADDR
    .apply(clean_addr)
    .apply(lambda x: x + ', Somerville, MA')
)

print(len(address_list))

address_list.to_csv('../data/address_MBL.csv')

# get list of addresses to download

In [ ]:
files_already_dowloaded = [f for f in listdir("../data/street_view_all/") 
                           if isfile(join("../data/street_view_all/", f))]

files_already_dowloaded[0]

is_row_downloaded_new_names = (
    address_list.SITE_ADDR
    .apply(clean_address)
    .apply(lambda x: x in files_already_dowloaded)
)
print(sum(is_row_downloaded_new_names))

is_row_downloaded_old_names = (
    address_list.SITE_ADDR
    .apply(lambda x: clean_address(x, slugify = False))
    .apply(lambda x: x in files_already_dowloaded)
)
print(sum(is_row_downloaded_old_names))

address_list = address_list[
    ~ is_row_downloaded_old_names & ~ is_row_downloaded_new_names
]

address_list

# Download Images

In [ ]:
def check_for_file(dir_path, image_name):
    files_already_dowloaded = [f for f in listdir(dir_path) if isfile(join(dir_path, f))]
    return image_name in files_already_dowloaded

In [ ]:
for idx, address in enumerate(address_list.SITE_ADDR):
    street_view_all = "../data/street_view_all/"
    image_name = clean_address(address)
    filename = street_view_all + image_name
    save_streetview_image(address, filename)
    
    file_exists = (
        check_for_file(street_view_all, clean_address(address)) 
        or check_for_file(street_view_all, clean_address(address, slugify = False))
    )
    if not file_exists:
        break
        
    if idx % 10 == 0:
        print(f"{idx+1}/{len(address_list)} images downloaded")